In [1]:
#import packages

import numpy as np
import pandas as pd
import random
import pickle
import itertools
import folium #mapping
import seaborn as sns #color palette
#from colormap import rgb2hex #chnage RGB codes to hex codes

In [65]:
## USE THIS CODE TO IMPORT DATA

file_name = '../data/results/results_pooled_cement_only.pkl' ##results_toydata_uniform.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

In [66]:
#convert to dataframe

df = pd.DataFrame(loaded_list, columns = ['supplier', 'receiver', 'distance', 'volume'])
df.head()

#convert suppliers to list to get rid of weird numpy object datatype
supplier_split = df.supplier.to_list()
receiver_split = df.receiver.to_list()
#supplier_split[0:5]


#convert suppliers to lat and lon
supplier_lat= [supplier_split[j][0] for j in range(0,len(supplier_split))]
supplier_lon = [supplier_split[j][1] for j in range(0,len(supplier_split))]

#convert to list of receiver lat and long
receiver_lat= [receiver_split[j][0] for j in range(0,len(receiver_split))]
receiver_lon = [receiver_split[j][1] for j in range(0,len(receiver_split))]

#add back into the df
df['supplier_lat'] = supplier_lat
df['supplier_lon'] = supplier_lon
df['receiver_lat'] = receiver_lat
df['receiver_lon'] = receiver_lat

df.head()

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[34.6222, -117.1001]","[38.773042, -121.863128]",389.583778,12834.300000,34.6222,-117.1001,38.773042,38.773042
1,"[37.3181, -122.091]","[33.91736204, -117.8454482]",334.721870,88.300192,37.3181,-122.0910,33.917362,33.917362
2,"[37.3181, -122.091]","[33.90629635, -117.8173157]",336.397606,164.733310,37.3181,-122.0910,33.906296,33.906296
3,"[37.3181, -122.091]","[33.87178115, -117.8282233]",337.664318,211.866490,37.3181,-122.0910,33.871781,33.871781
4,"[40.7369, -122.3223]","[38.14384789, -121.7042643]",181.914747,1072.200000,40.7369,-122.3223,38.143848,38.143848


In [67]:
#create a color palette, repeat it 100 times to see how many things I can visualize at once in the html
palette = ['red', 'green', 'purple', 'orange',  'darkblue',
           'gray','cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
           'black','lightgray','lightred','blue','beige']*100
            #‘lightgreen’] ‘gray’, ‘black’, ‘lightgray’] 'darkred','lightred','blue','beige','white',


#palette

## Map Everything

Shape of dataframe

In [68]:
df.shape

(9, 8)

In [69]:
#view a record
df[df['supplier_lat'] == 35.399380]
df

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[34.6222, -117.1001]","[38.773042, -121.863128]",389.583778,12834.300000,34.622200,-117.100100,38.773042,38.773042
1,"[37.3181, -122.091]","[33.91736204, -117.8454482]",334.721870,88.300192,37.318100,-122.091000,33.917362,33.917362
2,"[37.3181, -122.091]","[33.90629635, -117.8173157]",336.397606,164.733310,37.318100,-122.091000,33.906296,33.906296
3,"[37.3181, -122.091]","[33.87178115, -117.8282233]",337.664318,211.866490,37.318100,-122.091000,33.871781,33.871781
4,"[40.7369, -122.3223]","[38.14384789, -121.7042643]",181.914747,1072.200000,40.736900,-122.322300,38.143848,38.143848
5,"[34.437557, -116.891034]","[38.773042, -121.863128]",407.017085,421.700000,34.437557,-116.891034,38.773042,38.773042
6,"[34.819863, -118.748732]","[34.31486289, -119.318594]",47.618948,2476.800000,34.819863,-118.748732,34.314863,34.314863
7,"[35.029298, -118.316236]","[35.612471, -119.726892]",89.268250,712.000000,35.029298,-118.316236,35.612471,35.612471
8,"[34.6045, -117.3382]","[38.64352562, -121.6411539]",367.002851,270.500000,34.604500,-117.338200,38.643526,38.643526


Get counts by supplier

In [70]:
#counts by each supplier
df_counts = df.groupby(['supplier_lat','supplier_lon']).count().reset_index()
df_sorted = df_counts.sort_values('distance')
df_sorted.tail(20)

,supplier_lat,supplier_lon,supplier,receiver,distance,volume,receiver_lat,receiver_lon
0,34.437557,-116.891034,1,1,1,1,1,1
1,34.604500,-117.338200,1,1,1,1,1,1
2,34.622200,-117.100100,1,1,1,1,1,1
3,34.819863,-118.748732,1,1,1,1,1,1
4,35.029298,-118.316236,1,1,1,1,1,1
6,40.736900,-122.322300,1,1,1,1,1,1
5,37.318100,-122.091000,3,3,3,3,3,3


In [71]:
df = df[df['volume'] != 12834.30000]
df

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
1,"[37.3181, -122.091]","[33.91736204, -117.8454482]",334.721870,88.300192,37.318100,-122.091000,33.917362,33.917362
2,"[37.3181, -122.091]","[33.90629635, -117.8173157]",336.397606,164.733310,37.318100,-122.091000,33.906296,33.906296
3,"[37.3181, -122.091]","[33.87178115, -117.8282233]",337.664318,211.866490,37.318100,-122.091000,33.871781,33.871781
4,"[40.7369, -122.3223]","[38.14384789, -121.7042643]",181.914747,1072.200000,40.736900,-122.322300,38.143848,38.143848
5,"[34.437557, -116.891034]","[38.773042, -121.863128]",407.017085,421.700000,34.437557,-116.891034,38.773042,38.773042
6,"[34.819863, -118.748732]","[34.31486289, -119.318594]",47.618948,2476.800000,34.819863,-118.748732,34.314863,34.314863
7,"[35.029298, -118.316236]","[35.612471, -119.726892]",89.268250,712.000000,35.029298,-118.316236,35.612471,35.612471
8,"[34.6045, -117.3382]","[38.64352562, -121.6411539]",367.002851,270.500000,34.604500,-117.338200,38.643526,38.643526


Show the 10 suppliers with the shortest mean distance

In [72]:
#group the dataframe by mean distance
df_grouped = df.groupby(['supplier_lat','supplier_lon']).mean('distance').reset_index()
df_grouped

,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
0,34.437557,-116.891034,407.017085,421.700000,38.773042,38.773042
1,34.604500,-117.338200,367.002851,270.500000,38.643526,38.643526
2,34.819863,-118.748732,47.618948,2476.800000,34.314863,34.314863
3,35.029298,-118.316236,89.268250,712.000000,35.612471,35.612471
4,37.318100,-122.091000,336.261265,154.966664,33.898480,33.898480
5,40.736900,-122.322300,181.914747,1072.200000,38.143848,38.143848


In [73]:
#sort on distance
df_grouped = df_grouped.sort_values('distance')
df_grouped

,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
2,34.819863,-118.748732,47.618948,2476.800000,34.314863,34.314863
3,35.029298,-118.316236,89.268250,712.000000,35.612471,35.612471
5,40.736900,-122.322300,181.914747,1072.200000,38.143848,38.143848
4,37.318100,-122.091000,336.261265,154.966664,33.898480,33.898480
1,34.604500,-117.338200,367.002851,270.500000,38.643526,38.643526
0,34.437557,-116.891034,407.017085,421.700000,38.773042,38.773042


In [74]:
#cut off at the top 10
df_grouped = df_grouped.reset_index() #10 at the .tail(10) head(10)
df_grouped

,index,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
0,2,34.819863,-118.748732,47.618948,2476.800000,34.314863,34.314863
1,3,35.029298,-118.316236,89.268250,712.000000,35.612471,35.612471
2,5,40.736900,-122.322300,181.914747,1072.200000,38.143848,38.143848
3,4,37.318100,-122.091000,336.261265,154.966664,33.898480,33.898480
4,1,34.604500,-117.338200,367.002851,270.500000,38.643526,38.643526
5,0,34.437557,-116.891034,407.017085,421.700000,38.773042,38.773042


In [75]:
df_small = pd.DataFrame()

#loop through and get the full dataset. can't group by the lat/long because it's some unhashable type.
for i in range(0,df_grouped.shape[0]):
    this_lat = df_grouped['supplier_lat'][i]
    this_lon = df_grouped['supplier_lon'][i]
    receivers = df[(df['supplier_lat'] == this_lat) & (df['supplier_lon'] == this_lon)]
    df_small = df_small.append(receivers)
    
df_small.shape

(8, 8)

In [76]:
#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers.shape
#unique_suppliers['supplier_lon'][0]

(6, 2)

In [77]:
vols = df['volume']
sum(vols)

5418.099992

In [78]:
#determine the maximum volume - this will be used to scale the line weight
volumes = df['volume']
max_volume = np.max(volumes)
max_volume



2476.8

In [79]:
volumes = df['volume']
quantiles = np.quantile(volumes, [0.2,0.4,0.6,0.8])
quantiles
volumes

1      88.300192
2     164.733310
3     211.866490
4    1072.200000
5     421.700000
6    2476.800000
7     712.000000
8     270.500000
Name: volume, dtype: float64

In [80]:
#create map

#center the map in Cali
#mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=20 )
receiver_count = 0
 
#plot all the suppliers 
latlon = df_small['supplier']
mapit = folium.Map( location=[37.3427, -119.2244], zoom_start=6)

#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers

#loop through suppliers, paint all the receivers in the same color
for i in range(0,unique_suppliers.shape[0]):
    
    #big circle for supplier
    folium.CircleMarker( location=[ unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i] ], fill=True, 
                        fill_color=palette[i], color=palette[i],  radius=10 ).add_to( mapit ) #'
    
    #find all receivers associated with the supplier late and lon, filter the df
    this_lat = unique_suppliers['supplier_lat'][i]
    this_lon = unique_suppliers['supplier_lon'][i]
    receivers = df_small[(df_small['supplier_lat'] == this_lat) & (df_small['supplier_lon'] == this_lon)]
    
    #plot the receivers with a new color each time
    latlon = receivers['receiver']
    #print(receivers.shape[0])
    #print(receivers.shape[1])
    receiver_count += receivers.shape[0] #verify 1000
    
    #for each receiver  
    for j in range(0,len(latlon)): #coord in latlon: 
        
        lat = latlon.iloc[j][0]
        lon = latlon.iloc[j][1]
        
        volume = receivers.iloc[j]['volume']
        #if volume > quantiles[3]:
        #    quantile = 1 #80-100 percentile
        #elif volume > quantiles[2]:
        #    quantile = 0.8 #60-80 percentile
        #elif volume > quantiles[1]:
        #    quantile = 0.6 #40-60 percentile
        #elif volume > quantiles[0]:
        #    quantile = 0.4 #20-40 percentile
        #else:
        #    quantile  = 0.2 #0-20 percentile
        #line_weight = quantile * 5
        line_weight = volume/max_volume * 20

        #print(line_weight)
        #print(volume, quantile)
        
        #line coordinates
        line_coords = [ [unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i]],
                      [lat, lon] ]#[coord[0], coord[1] ]  ]
        my_PolyLine= folium.PolyLine(locations=line_coords,color=palette[i],weight=line_weight) #color and supplier use i index
        mapit.add_child(my_PolyLine)
        
        #dot for the receiver
        folium.CircleMarker( location=[ lat,lon ], fill=True, fill_color=palette[i], color=palette[i],  radius=line_weight ).add_to( mapit )

print(receiver_count)
mapit

mapit.save(outfile= "visualization.html")

8
